In [ ]:
import numpy as np
import pandas as pd
from simple.ohlc import renko, midPrice
from simple.plotly import chartFigure
from simple.funcs import symlog
from simple.backtest import npBacktestIOC

In [ ]:
T = np.load('data/ETHUSDT.2022-05-08.tick.npz')['ETHUSDT'].view(np.recarray)
len(T)

In [ ]:
Buy = T[T.VolumeA > 0]
Sell = T[T.VolumeA < 0]
Buy[:5]

In [ ]:
S = np.load('data/ETHUSDT.2022-05-08.bidask.npz')['ETHUSDT'].view(np.recarray)
S[:5]

In [ ]:
len(T), len(S)

In [ ]:
C = renko(T, 0.5)
C[:5]

In [ ]:
len(C)

In [ ]:
signal = np.zeros(len(S.DT))
for c in C:
    tm = T.DateTimeA[c[1]] - T.DateTimeA[c[0]]
    if tm < np.timedelta64(10000, 'μs'):
        k = np.searchsorted(S.DT, T.DateTimeA[c[1]])
        signal[k] = 1 if c[3] > c[2] else - 1
        print(c, tm, k)

In [ ]:
trades = npBacktestIOC(S.DT.astype(np.int64), S.A, S.B, signal, 0)
trades